In [1]:
import pandas as pd
import numpy as np
import re, reprlib, sys
import matplotlib.pyplot as plt
from itertools import *
import random as ran
from logging import *
from pprint import *
from time import *

ran.seed(5125)

%load_ext cython
%load_ext cythonmagic
%matplotlib inline
%matplotlib notebook

from pandas_confusion import *
from scipy.cluster.vq import *

import nltk as n
import nltk, nltk.classify.util, nltk.metrics, nltk.tokenize, nltk.stem
from nltk.corpus import *
from nltk.stem import *
from nltk.classify import *
from nltk.collocations import *
from nltk.metrics import BigramAssocMeasures as BAM
from nltk.probability import *
from nltk.classify.scikitlearn import *
from nltk.tag.sequential import *
from nltk.tag import *
from nltk.tag.util import *


from sklearn_pandas import *

import sklearn as sk
from sklearn import *
from sklearn.feature_extraction.text import *
from sklearn.svm import *
from sklearn.cross_validation import *
from sklearn.pipeline import *
from sklearn.multiclass import *
from sklearn.datasets import *
from sklearn.naive_bayes import *
from sklearn.neighbors import *
from sklearn.feature_selection import *
from sklearn.ensemble import *
from sklearn.linear_model import *
from sklearn.tree import *
from sklearn.grid_search import *
from sklearn.base import *
from sklearn.datasets.twenty_newsgroups import *
from sklearn.decomposition import *
from sklearn.feature_extraction import *
from sklearn.metrics import *
# n.download()

/usr/local/lib/python3.4/dist-packages/IPython/extensions/cythonmagic.py:21: UserWarning: The Cython magic has been moved to the Cython package
  warnings.warn("""The Cython magic has been moved to the Cython package""")
/usr/local/lib/python3.4/dist-packages/sklearn/lda.py:4: DeprecationWarning: lda.LDA has been moved to discriminant_analysis.LinearDiscriminantAnalysis in 0.17 and will be removed in 0.19
  "in 0.17 and will be removed in 0.19", DeprecationWarning)
/usr/local/lib/python3.4/dist-packages/sklearn/qda.py:4: DeprecationWarning: qda.QDA has been moved to discriminant_analysis.QuadraticDiscriminantAnalysis in 0.17 and will be removed in 0.19.
  "in 0.17 and will be removed in 0.19.", DeprecationWarning)


### R script adding new features to dataset 

In [ ]:
# http://www.mango-solutions.com/wp/2015/10/integrating-python-and-r-part-ii-executing-r-from-python-and-vice-versa/
command = 'Rscript'
path2script = 'lexical_features.R'

# Build subprocess command
cmd = [command, path2script]

subprocess.check_output(cmd, universal_newlines=True)

In [5]:
data_n = pd.read_csv("../raw_data/data_n.csv", parse_dates=True, infer_datetime_format=True)
data_n.head()

,Unnamed: 0,#AUTHID,STATUS,sEXT,sNEU,sAGR,sCON,sOPN,cEXT,cNEU,...,Number_of_Semicolons,Number_of_Colons,Average_Word_Length,POS_sentiment,NEG_sentiment,OverAll_sentiment,Lexical_Diversity,Number_of_FunctionalWords,Number_of_Pronouns,Number_of_PROPNAMEs
0,1,b7b7764cfa1c523e4e93ab2a79a946c4,likes the sound of thunder.,2.65,3,3.15,3.25,4.4,n,y,...,0,0,4.400,0,0,0,0.700,2,0,0
1,2,b7b7764cfa1c523e4e93ab2a79a946c4,is so sleepy it's not even funny that's she ca...,2.65,3,3.15,3.25,4.4,n,y,...,0,0,3.615,0,0,0,0.577,10,1,0
2,3,b7b7764cfa1c523e4e93ab2a79a946c4,is sore and wants the knot of muscles at the b...,2.65,3,3.15,3.25,4.4,n,y,...,0,0,3.500,0,2,-2,0.509,14,1,0
3,4,b7b7764cfa1c523e4e93ab2a79a946c4,likes how the day sounds in this new song.,2.65,3,3.15,3.25,4.4,n,y,...,0,0,3.667,0,0,0,0.611,4,0,0
4,5,b7b7764cfa1c523e4e93ab2a79a946c4,is home. <3,2.65,3,3.15,3.25,4.4,n,y,...,0,0,2.333,0,0,0,0.857,1,0,0


### Replace 'y' and 'n' with 1 & 0

In [8]:
# http://stackoverflow.com/a/17702781
d = {'n': 0, 'y': 1}
data_n = data_n.replace(d)
data_n.head(n=1)

,Unnamed: 0,#AUTHID,STATUS,sEXT,sNEU,sAGR,sCON,sOPN,cEXT,cNEU,...,Number_of_Semicolons,Number_of_Colons,Average_Word_Length,POS_sentiment,NEG_sentiment,OverAll_sentiment,Lexical_Diversity,Number_of_FunctionalWords,Number_of_Pronouns,Number_of_PROPNAMEs
0,1,b7b7764cfa1c523e4e93ab2a79a946c4,likes the sound of thunder.,2.65,3,3.15,3.25,4.4,0,1,...,0,0,4.4,0,0,0,0.7,2,0,0


### Split stra. k-fold + selection of columns

In [10]:
# neu = data_n[["#AUTHID","STATUS","cNEU"]]
# train_neu, test_neu, y_train, y_test = sk.cross_validation.train_test_split(neu, neu["cNEU"], train_size = 0.66, stratify= neu["cNEU"])
# train_neu, test_neu, y_train, y_test 

In [ ]:
neu = data_n[["#AUTHID","STATUS","cNEU", "StringLength", 
                  "Number_of_Words", "Number_of_Dots", "Number_of_Commas", "Number_of_Semicolons", 
                  "Number_of_Colons", "Avarage_Word_Lenght", "Lexical_Diversity"]]
ext = data_n[["#AUTHID","STATUS","cEXT", "StringLength", 
                  "Number_of_Words", "Number_of_Dots", "Number_of_Commas", "Number_of_Semicolons", 
                  "Number_of_Colons", "Avarage_Word_Lenght", "Lexical_Diversity"]]
agr = data_n[["#AUTHID","STATUS","cAGR", "StringLength", 
                  "Number_of_Words", "Number_of_Dots", "Number_of_Commas", "Number_of_Semicolons", 
                  "Number_of_Colons", "Avarage_Word_Lenght", "Lexical_Diversity"]]
con = data_n[["#AUTHID","STATUS","cCON", "StringLength", 
                  "Number_of_Words", "Number_of_Dots", "Number_of_Commas", "Number_of_Semicolons", 
                  "Number_of_Colons", "Avarage_Word_Lenght", "Lexical_Diversity"]]
opn = data_n[["#AUTHID","STATUS","cOPN", "StringLength", 
                  "Number_of_Words", "Number_of_Dots", "Number_of_Commas", "Number_of_Semicolons", 
                  "Number_of_Colons", "Avarage_Word_Lenght", "Lexical_Diversity"]]

In [14]:
train_neu, test_neu = sk.cross_validation.train_test_split(neu, train_size = 0.66, stratify= neu["cNEU"])
train_ext, test_ext = sk.cross_validation.train_test_split(ext, train_size = 0.66, stratify= ext["cEXT"])
train_agr, test_agr = sk.cross_validation.train_test_split(agr, train_size = 0.66, stratify= agr["cAGR"])
train_con, test_con = sk.cross_validation.train_test_split(con, train_size = 0.66, stratify= con["cCON"])
train_opn, test_opn = sk.cross_validation.train_test_split(opn, train_size = 0.66, stratify= opn["cOPN"])

NameError: name 'ext' is not defined

In [ ]:
new_fea = data_n[["STATUS","cCON","cOPN","StringLength", 
                  "Number_of_Words", "Number_of_Dots", "Number_of_Commas", "Number_of_Semicolons", 
                  "Number_of_Colons", "Avarage_Word_Lenght", "Lexical_Diversity"]]
new_fea.head(n=1)

### Train + Test

### Confusion Matrix

In [ ]:
cm = ConfusionMatrix(data_n["cNEU"], y_pred)
cm.print_stats()